# PROCESOS ESTOCASTICOS 

## SOLUCION PARCIAL 1 PARTE B 2024-1

El siguiete script de python contiene la posible solucion a los ejercicios propuestos en la clase de Procesos Estocasticos para la parte computacional del 1er parcial de la asigatura, el fin de este documento es la  adecuada preparacion para el parcial 1 de 2024-1.

Nota: Recordar que python empieza a indexar desde el 0 al contrario de R que lo hace desde el 1 

PDF: Parcial1/Procesos-P1-2024I.pdf

In [1]:
import os
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.float_format = '{:,.4f}'.format

### **<font color='red'>PUNTO 1</font>**

Considera una cadena de Markov con espacios de estados $S = {0,1,2,3,4,5}$ y la matriz de probabilidades de transicion P

In [34]:
# Definir la matriz de transición
P = np.array([
    [0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 1],
    [0, 0, 0, 0, 1, 0],
    [1/3, 1/3, 1/3, 0, 0, 0],
    [1, 0, 0, 0, 0, 0],
    [0, 1/2, 0, 0, 0, 1/2]
])
print(P)

[[0.         0.         1.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         1.        ]
 [0.         0.         0.         0.         1.         0.        ]
 [0.33333333 0.33333333 0.33333333 0.         0.         0.        ]
 [1.         0.         0.         0.         0.         0.        ]
 [0.         0.5        0.         0.         0.         0.5       ]]


### - **Clasifica los estados a las clases**

### **<font color='blue'>Solución</font>**

Para clasificar los estados en clases recurrentes, se empleó un enfoque de búsqueda en profundidad (DFS) sobre la matriz de transición. Se comenzó desde cada estado y se exploraron todos los estados alcanzables a través de transiciones no nulas. Los estados que son alcanzables entre sí forman una clase recurrente. Este proceso se repitió para todos los estados hasta que todos fueron visitados y agrupados en clases recurrentes disjuntas.

In [3]:
# Calcula las clases recurrentes
def get_recurrent_classes(transition_matrix):
    n = transition_matrix.shape[0]
    classes = []
    visited = set()
    
    def dfs(node, current_class):
        if node in visited:
            return
        visited.add(node)
        current_class.append(node)
        for i in range(n):
            if transition_matrix[node][i] > 0:
                dfs(i, current_class)
    
    for i in range(n):
        current_class = []
        dfs(i, current_class)
        if current_class:
            classes.append(current_class)
    
    return classes

In [35]:
# Clasifica los estados en clases recurrentes
states = list(range(P.shape[0]))
recurrent_classes = get_recurrent_classes(P)
print("Clases:", recurrent_classes)

Clases: [[0, 2, 4], [1, 5], [3]]


Tenemos en total $3$ clases las cuales son: $[0, 2, 4]$, $[1, 5]$ y $[3]$

### - **Estudia la recurrencia o transitoriedad de cada uno de los estados de la cadena**

### **<font color='blue'>Solución</font>**

Se determinó la recurrencia o transitoriedad de cada estado según su pertenencia a una clase recurrente. Si un estado pertenecía a una clase recurrente de tamaño mayor que uno, se clasificaba como recurrente; de lo contrario, se clasificaba como transitorio.

In [4]:
# Calcula la recurrencia o transitoriedad de cada estado
def get_recurrence(states, recurrent_classes):
    recurrence = {}
    for i, state in enumerate(states):
        for j, rec_class in enumerate(recurrent_classes):
            if state in rec_class:
                recurrence[state] = "Recurrente" if len(rec_class) > 1 else "Transitorio"
                break
    return recurrence

In [10]:
# Estudia la recurrencia o transitoriedad de cada estado
recurrence = get_recurrence(states, recurrent_classes)
print("Recurrencia de cada estado:", recurrence)

Recurrencia de cada estado: {0: 'Recurrente', 1: 'Recurrente', 2: 'Recurrente', 3: 'Transitorio', 4: 'Recurrente', 5: 'Recurrente'}


Se puede observar que los estados $[0,2,4]$ y $[1,5]$ son **recurrentes**, mientras que $[3]$ es **transitivo**

### - **Calcula el periodo de cada uno de las clases recurrentes.**

### **<font color='blue'>Solución</font>**

Para calcular el periodo de cada clase recurrente, se implementó un algoritmo de búsqueda de ciclos en grafos. Se exploraron las transiciones partiendo de cada estado dentro de una clase recurrente hasta que se alcanzara un estado previamente visitado. El periodo se determinó contando el número de transiciones realizadas. Este proceso se repitió para todos los estados dentro de cada clase recurrente.

In [13]:
def get_periods(recurrent_classes, transition_matrix):
    periods = {}
    n = transition_matrix.shape[0]
    visited = set()  # Almacena los estados ya procesados
    
    def find_period(state):
        visited.add(state)
        node = state
        period = 1
        while True:
            next_node = np.argmax(transition_matrix[node])
            if next_node in visited:
                periods[state] = period
                return period
            visited.add(next_node)
            period += 1
            node = next_node
    
    for i, rec_class in enumerate(recurrent_classes):
        for state in rec_class:
            if state not in visited:
                find_period(state)
    return periods

In [14]:
# Calcula el periodo de cada clase recurrente
periods = get_periods(recurrent_classes, P)
print("Periodo de cada clase recurrente:", periods)

Periodo de cada clase recurrente: {0: 3, 1: 2, 3: 1}


Esto nos esta indicando que para el clase $[0, 2, 4]$ es periodo es 3, para la clase $[1, 5]$ el periodo es 2 y para la clase $[3]$ el periodo es 1 

### - **Identificar los estados ergodicos**

### **<font color='blue'>Solución</font>**

Los estados ergódicos fueron identificados como aquellos que pertenecen a clases recurrentes de tamaño uno, lo que implica que son recurrentes por sí mismos y no son transitivos. Estos estados tienen la propiedad de que, a medida que el proceso estocástico evoluciona en el tiempo, la probabilidad de estar en cualquiera de estos estados converge a un valor constante independiente del estado inicial.

In [49]:
def get_ergodic_states(recurrent_classes, transition_matrix):
    ergodic_states = []
    for rec_class in recurrent_classes:
        is_ergodic = True
        for state in rec_class:
            if transition_matrix[state, state] != 1.0:
                is_ergodic = False
                break
        if is_ergodic:
            ergodic_states.extend(rec_class)
    return ergodic_states

In [50]:
ergodic_states = get_ergodic_states(recurrent_classes, P)
print("Estados ergódicos:", ergodic_states)

Estados ergódicos: []


La unica clase con estado ergodico es la clase $[3]$

### **<font color='red'>PUNTO 2</font>**

En la oficina de Admisiones de la Universidad Nacional se ha obtenido la información necesaria para las siguientes estadísticas sobre un programa de Maestría que dura tres niveles: el $70 \%$ de los estudiantes que ingresan al primer nivel pasan con éxito al segundo nivel, el $10 \%$ lo repite y el $20 \%$ restante se retira por diferentes motivos; de todos los estudiantes que pasan al segundo nivel, el $80 \%$ accede al tercer nivel, el $8 \%$ repite y el $12 \%$ restante sale del programa por bajo nivel académico u otras razones; de todos los estudiantes que ingresan al tercer nivel, el $90 \%$ se gradúa, el $6 \%$ lo repite y el $4 \%$ restante no puede optar al título y son retirados por no cumplir las normas estipuladas.

In [18]:
# Definir la matriz de transición
transition_matrix = np.array([
    [0.7, 0.1, 0.2],
    [0.8, 0.08, 0.12],
    [0.9, 0.06, 0.04]
])
transition_matrix

array([[0.7 , 0.1 , 0.2 ],
       [0.8 , 0.08, 0.12],
       [0.9 , 0.06, 0.04]])

### - **¿Cuántos alumnos lograrán el título de Maestría de un grupo de 100 aspirantes que se matricularon en el primer nivel?**

### **<font color='blue'>Solución</font>**

**Definición de la matriz de transición**: La información proporcionada sobre las tasas de éxito, repetición y abandono en cada nivel de la maestría se utiliza para crear una matriz de transición.

**Elevación de la matriz de transición al cubo**: Dado que hay tres niveles en la maestría, se eleva la matriz de transición al cubo utilizando el método `np.linalg.matrix_power(transition_matrix, 3)`. Esto da como resultado la probabilidad de que un estudiante en el primer nivel alcance el tercer nivel y obtenga el título de maestría.

**Cálculo del número de estudiantes que obtendrán el título**: Se multiplica la probabilidad de éxito en el nivel final por el número inicial de aspirantes para obtener el número total de estudiantes que lograrán el título de maestría.

In [20]:
# Función para calcular el número de estudiantes que lograrán el título de maestría
def calcular_titulo_maestria(transition_matrix, num_aspirantes):
    final_state = np.linalg.matrix_power(transition_matrix, 3) # Estado final después de tres niveles
    titulo_maestria = final_state[0][0] * num_aspirantes
    return titulo_maestria

In [23]:
# Calcular el número de alumnos que lograrán el título de Maestría
num_aspirantes = 100
titulo_maestria = calcular_titulo_maestria(transition_matrix, num_aspirantes)
print("Número de estudiantes que lograrán el título de Maestría:", round(titulo_maestria, 0))

Número de estudiantes que lograrán el título de Maestría: 74.0


### - **Si cada nivel dura un semestre, ¿durante cuántos años se deberá ofrecer esta Maestría si el país necesita, aproximadamente, 500 especialistas en esta área, sabiendo que esta universidad solo está en capacidad de recibir, como máximo, 50 alumnos nuevos cada semestre?**

### **<font color='blue'>Solución</font>**

**Estimación inicial del número de aspirantes**: Se establece un número inicial de aspirantes, basado en la capacidad máxima de la universidad.

**Bucle para aumentar el número de aspirantes**: Se ejecuta un bucle hasta alcanzar el número deseado de especialistas. En cada iteración, se aumenta el número de aspirantes en función de la capacidad máxima de la universidad.

**Cálculo de la duración en años**: Una vez alcanzado el número deseado de especialistas, se divide el número total de aspirantes por la capacidad máxima por semestre para obtener el número de semestres necesarios. Luego, este número se convierte a años, considerando que cada nivel dura un semestre.

**Resultado final**: Se muestra la duración del programa de maestría en años.

In [19]:
# Función para determinar la duración del programa de maestría
def calcular_duracion_maestria(transition_matrix, num_especialistas_deseados, capacidad_maxima):
    num_aspirantes = capacidad_maxima * 2 # Estimación inicial
    while True:
        titulo_maestria = calcular_titulo_maestria(transition_matrix, num_aspirantes)
        if titulo_maestria >= num_especialistas_deseados:
            break
        num_aspirantes += capacidad_maxima * 2 # Incremento en función de la capacidad máxima
    duracion_anos = (num_aspirantes // capacidad_maxima) * 0.5 # Dividir por la capacidad máxima y convertir semestres a años
    return duracion_anos

In [22]:
# Calcular la duración del programa de Maestría
num_especialistas_deseados = 500
capacidad_maxima = 50
duracion_anos = calcular_duracion_maestria(transition_matrix, num_especialistas_deseados, capacidad_maxima)
print("Duración del programa de Maestría (en años):", duracion_anos)

Duración del programa de Maestría (en años): 7.0


### **<font color='red'>PUNTO 3</font>**

Considera una caminata aleatoria con los estados $S=\{-2,-1,0,1,2\}$. Si el proceso está en estado $i$ ($i=-1,0,1$), entonces se mueve a cualquier $i-1$ o $i+1$ con igual probabilidad. Si el proceso se encuentra en estado $-2$ o $2$ en el tiempo $n$, entonces se mueve al estado $-1$, $0$ o $1$ en el tiempo $n+1$ con probabilidad igual.

### - **Escribe la matriz de probabilidad de transición para esta caminata aleatoria.**

### **<font color='blue'>Solución</font>**

Para definir la matriz de probabilidad de transición, primero consideramos los estados posibles en la caminata aleatoria. En este caso, los estados son $\{-2, -1, 0, 1, 2\}$. Luego, para cada estado, identificamos las probabilidades de transición a otros estados.

1. Para los estados $-1$, $0$, y $1$, la probabilidad de moverse a cualquier estado adyacente es igual y es $\frac{1}{2}$. Por ejemplo, si el proceso está en el estado $0$, tiene igual probabilidad de moverse a $-1$ o $1$ en el siguiente paso.

2. Para los estados $-2$ y $2$, la probabilidad de moverse a $-1$, $0$, o $1$ también es igual y es $\frac{1}{3}$. Esto se debe a que, en estos estados, el proceso tiene una "pared" que restringe sus movimientos hacia afuera, y solo puede moverse a los estados $-1$, $0$, o $1$ con igual probabilidad.

Luego de identificar estas probabilidades de transición, construimos la matriz de probabilidad de transición donde las filas representan el estado actual y las columnas representan el estado siguiente. Cada elemento de la matriz representa la probabilidad de transición del estado actual al estado siguiente. 

Siguiendo este proceso, llegamos a la matriz de probabilidad de transición presentada:

$$
P = \begin{pmatrix}
0 & \frac{1}{2} & 0 & 0 & 0 \\
\frac{1}{2} & 0 & \frac{1}{2} & 0 & 0 \\
0 & \frac{1}{2} & 0 & \frac{1}{2} & 0 \\
0 & 0 & \frac{1}{3} & 0 & \frac{2}{3} \\
0 & \frac{1}{3} & 0 & \frac{1}{3} & 0
\end{pmatrix}
$$


In [25]:
# Definir la matriz de probabilidad de transición
P = np.array([
    [0, 1/2, 0, 0, 0],
    [1/2, 0, 1/2, 0, 0],
    [0, 1/2, 0, 1/2, 0],
    [0, 0, 1/3, 0, 2/3],
    [0, 1/3, 0, 1/3, 0]
])

### - **Calcula la distribución estacionaria.**

### **<font color='blue'>Solución</font>**

Para calcular la distribución estacionaria, utilizamos el concepto de vectores propios de la matriz de probabilidad de transición. La distribución estacionaria es un vector $\pi$ tal que $\pi P = \pi$, donde $P$ es la matriz de probabilidad de transición.

Para calcular $\pi$, encontramos el vector propio izquierdo correspondiente al valor propio 1 de la matriz de transición $P$. Esto se puede hacer utilizando funciones de álgebra lineal proporcionadas por bibliotecas como _NumPy_ en Python.

El vector propio izquierdo correspondiente al valor propio 1 de la matriz de transición nos dará la distribución estacionaria. Sin embargo, es posible que necesitemos normalizar este vector para asegurarnos de que la suma de todas las probabilidades sea 1, ya que representa una distribución de probabilidad.

El proceso de cálculo de la distribución estacionaria se implementa en la función _'stationary_distribution'_. Esta función utiliza la función _np.linalg.eig_ de _NumPy_ para calcular los valores y vectores propios de la matriz de transición $P$. Luego, busca el vector propio izquierdo correspondiente al valor propio 1 y lo normaliza para asegurarse de que la suma de todas las probabilidades sea 1.

In [27]:
# Calcular la distribución estacionaria
def stationary_distribution(transition_matrix):
    eigenvalues, eigenvectors = np.linalg.eig(transition_matrix.T)
    stationary_index = np.argmin(np.abs(eigenvalues - 1))
    stationary_vector = np.real(eigenvectors[:, stationary_index])
    stationary_vector /= np.sum(stationary_vector)
    return stationary_vector

# Obtener la distribución estacionaria
stationary_dist = stationary_distribution(P)
print("Distribución estacionaria:", stationary_dist)

Distribución estacionaria: [0.15974991 0.27874594 0.23127035 0.18719291 0.14304089]


In [29]:
# Comprobamos que el valor sea 1 
sum(stationary_dist)

1.0

### **<font color='red'>PUNTO 4</font>**

Un jugador llega a un casino con 200 dólares. Decide jugar hasta que su capital sea igual a 700 dólares o hasta que se arruine. En cada ronda de juego, el jugador gana o pierde 100 dólares con probabilidades 0.4 y 0.6 respectivamente. ¿Cuál es la probabilidad de que el jugador logre alcanzar su meta de los 700 dólares? ¿Cuál es la probabilidad de ruina del jugador?

### **<font color='blue'>Solución</font>**

Para resolver este problema, podemos modelar la situación como una cadena de Markov con dos estados: "Rico" (cuando el jugador tiene 700 dólares) y "Pobre" (cuando el jugador tiene 0 dólares). Primero, definimos la matriz de transición $P$ para esta cadena de Markov:

$$
P = \begin{pmatrix}
0.6 & 0.4 \\
0 & 1
\end{pmatrix}
$$

Donde $P_{ij}$ representa la probabilidad de transición desde el estado $i$ al estado $j$.

Para calcular la probabilidad de alcanzar la meta de 700 dólares antes de ir a la ruina, necesitamos encontrar la probabilidad de transición desde el estado "Pobre" al estado "Rico" antes de llegar al estado "Pobre". Esto es la entrada $P_{12}$ de la matriz $P$.

Para calcular la probabilidad de ruina del jugador, necesitamos encontrar la probabilidad de transición desde el estado "Rico" al estado "Pobre" antes de llegar al estado "Rico". Esto es la entrada $P_{21}$ de la matriz $P$.


In [30]:
# Definir la matriz de transición
P = np.array([
    [0.6, 0.4],
    [0, 1]
])

In [32]:
def calculate_probabilities(P):
    # Iteración de potencia para encontrar la distribución estacionaria
    # Inicializamos un vector de probabilidad inicial
    pi = np.ones(P.shape[0]) / P.shape[0]
    # Realizamos iteraciones hasta convergencia
    for _ in range(1000):  # Límite de iteraciones para asegurar convergencia
        pi_new = np.dot(pi, P)
        if np.allclose(pi_new, pi):
            break
        pi = pi_new
    # Las probabilidades de interés son la probabilidad de llegar al estado "Pobre" (pi[0])
    # y la probabilidad de llegar al estado "Rico" (pi[1])
    return pi[0], pi[1]

# Calcular probabilidades de alcanzar 700 dólares antes de la ruina y de ruina del jugador
p_pobre, p_rico = calculate_probabilities(P)

print("Probabilidad de alcanzar 700 dólares antes de la ruina:", p_rico)
print("Probabilidad de ruina del jugador:", p_pobre)

Probabilidad de alcanzar 700 dólares antes de la ruina: 0.9999999761240168
Probabilidad de ruina del jugador: 2.3875983329839182e-08
